# Local Function Review

In [24]:
def outer_func():
    message = "Hi"
    def inner_func():
        print(message)
        
    return inner_func()

In [25]:
# Test it
outer_func()

Hi


In [13]:
def outer_func():
    message = "Hi"
    def inner_func():
        print(message)
        
    return inner_func

In [15]:
my_func = outer_func()
my_func

<function __main__.outer_func.<locals>.inner_func>

In [16]:
my_func()

Hi


In [17]:
# Biw add asine avruabkes
def outer_func(msg):
    message = msg
    def inner_func():
        print(message)
        
    return inner_func

In [22]:
hi_func = outer_func("Hi")
bye_func = outer_func("Bye")

#run it
hi_func()
bye_func()

Hi
Bye


In [26]:
# Biw add asine avruabkes
def outer_func(msg):
    def inner_func():
        print(msg)
        
    return inner_func

In [27]:
hi_func = outer_func("Hi")
bye_func = outer_func("Bye")

#run it
hi_func()
bye_func()

Hi
Bye


## Decorators
- A function that takes another function as a argument
- Adds some functionality
- Returns another function
- All of the with altering the original function

In [32]:
def decorator_func(original_func):
    def wrapper_func():
        return original_func()
    return wrapper_func

def display():
    print("Display function ran")

In [34]:
# Test it
decorated_display = decorator_func(display)
decorated_display

<function __main__.decorator_func.<locals>.wrapper_func>

In [35]:
decorated_display()

Display function ran


In [38]:
def decorator_func(original_func):
    def wrapper_func():
        print("Wrapper executed this before {}".format(original_func.__name__))
        return original_func()
    return wrapper_func

def display():
    print("Display function ran")

In [39]:
# Test it
decorated_display = decorator_func(display)
decorated_display()

Wrapper executed this before display
Display function ran


## Now decorate your functions
**@Decorator function name**

In [45]:
def decorator_func(original_func):
    def wrapper_func():
        print("Wrapper executed this before {}".format(original_func.__name__))
        return original_func()
    return wrapper_func

@decorator_func
def display():
    print("Display function ran")

In [46]:
# Test it
display()

Wrapper executed this before display
Display function ran


This will not work if our original function takes arguments

In [51]:
@decorator_func
def display_info(name, age):
    print("Display _info ran with arguments ({}, {})".format(name, age))

In [52]:
display_info("Mario", 51)

TypeError: wrapper_func() takes 0 positional arguments but 2 were given

We can fix this with **\*args** and **\*\*kwargs**

In [53]:
def decorator_func(original_func):
    def wrapper_func(*args, **kwargs):
        print("Wrapper executed this before {}".format(original_func.__name__))
        return original_func(*args, **kwargs)
    return wrapper_func

@decorator_func
def display():
    print("Display function ran")
    
@decorator_func
def display_info(name, age):
    print("Display _info ran with arguments ({}, {})".format(name, age))

In [55]:
display()
display_info("Mark", 51)

Wrapper executed this before display
Display function ran
Wrapper executed this before display_info
Display _info ran with arguments (Mark, 51)


### Example: escape_unicode

In [60]:
def escape_unicode(f):
    def wrap(*args, **kwargs):
        return ascii(f(*args, **kwargs))
    return wrap

@escape_unicode
def mexico_city():
    # Alt + 130 for é
    return "México"

In [61]:
mexico_city()

"'M\\xe9xico'"

### What can be a decorator?
- class Objects the must be callable with the **dunder call**
- Functions as decorators

#### Classes as Decorators

In [81]:
class DecoratorClass(object):
    def __init__(self, original_f):
        print("1")
        self.original_f = original_f
        
    def __call__(self, *args, **kwargs):
        print("call method executed this before {}".format(self.original_f.__name__))
        return self.original_f(*args, **kwargs)
    
@DecoratorClass
def display():
    print("Display function ran")
    
@DecoratorClass
def display_info(name, age):
    print("Display _info ran with arguments ({}, {})".format(name, age))

1
1


In [82]:
display()
display_info("Mark", 51)

call method executed this before display
Display function ran
call method executed this before display_info
Display _info ran with arguments (Mark, 51)


### Instances as Decorators

A class instande

In [89]:
class Trace:
    def __init__(self):
        self._enable = True
        
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self._enable:
                print("Calling {}".format(f.__name__))
            return f(*args, **kwargs)
        return wrap
    
# Create an instance
tracer = Trace()

# Instance as decorator
@tracer
def rotate_list(l):
    return l[1:] + [l[0]]


Unlike our previous example the **class object itself is not the decorator**,  Rather, instances of trace can be used as decorators.

In [90]:
# test it
l1 = [1, 2, 3]
l1 = rotate_list(l1)
l1

Calling rotate_list


[2, 3, 1]

In [91]:
# turn off the enable key from the instance
tracer._enable = False
l1 = [1, 2, 3]
l1 = rotate_list(l1)
l1

[2, 3, 1]

## Back to function decortors


In [104]:
import time

def my_timer(f):
    """
    Helps you keep track of the loggin part of a function
    """
    def wrap(*args, **kwargs):
        t1 = time.time()
        result = f(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(f.__name__, t2))
        return result
    return wrap

# Test it
@my_timer
def display_info(name, age):
    time.sleep(5)
    print("display_info ran with arguments ({}, {})".format(name, age))

In [105]:
# test it
display_info("Mark", 51)

display_info ran with arguments (Mark, 51)
display_info ran in: 5.014963388442993


## Multiple decators

In [118]:
def my_logger(f):
    import logging
    logging.basicConfig(filename="{}.log".format(f.__name__), level=logging.INFO)
    def wrap(*args, **kwargs):
        logging.info("Ran with args: {}, and kwargs: {}".format(args, kwargs))
        return f(*args, **kwargs)
    return wrap


@my_timer
@my_logger
def display_info(name, age):
    time.sleep(1)
    print("display_info ran with arguments ({}, {})".format(name, age))
    
# Test it
display_info("Weber", 125)

display_info ran with arguments (Weber, 125)
wrap ran in: 1.0014781951904297


In [119]:
# switch order of decorators
# muultiple decotars
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("display_info ran with arguments ({}, {})".format(name, age))
    
# Test it
display_info("Weber", 125)

display_info ran with arguments (Weber, 125)
display_info ran in: 1.0016953945159912


Wrap everything

In [132]:
import time
from functools import wraps

def my_logger(f):
    import logging
    logging.basicConfig(filename="{}.log".format(f.__name__), level=logging.INFO)
    @wraps(f)
    def wrap(*args, **kwargs):
        logging.info("Ran with args: {}, and kwargs: {}".format(args, kwargs))
        return f(*args, **kwargs)
    return wrap

def my_timer(f):
    @wraps(f)
    def wrap(*args, **kwargs):
        t1 = time.time()
        result = f(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}".format(f.__name__, t2))
        return result
    return wrap


@my_logger
@my_timer
def display_info(name, age, **kwargs):
    time.sleep(1)
    print("display_info ran with arguments ({}, {})".format(name, age))
    
# Test it
display_info("Weber", 125, k="Hello")

display_info ran with arguments (Weber, 125)
display_info ran in: 1.0001111030578613
